In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from time import sleep
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import csv

In [2]:
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [ ]:
url = 'https://www.avito.ma/fr/maroc/appartements-%C3%A0_vendre?o='
options = webdriver.ChromeOptions()
options.add_argument('--inconginto')
driver = webdriver.Chrome(options=options)

path_to_file = "Appartements2.csv"
csvFile = open(path_to_file, 'w', newline='', encoding="utf-8")
csvWriter = csv.writer(csvFile)

csvWriter.writerow(["Titre", "Prix", "Ville" , "Type", "Secteur", "Etage", "Age du bien", "Surface habitable", "NB Salons", "NB Chambres", "NB Salles de Bain", "Frais de syndic / mois", "Description"])

## Boucle 1 permet de parcourir 100 page chaque page a 35 appartement
for i in range(100, 950):
    
    driver.get(url+str(i))

    try:
        app_link = driver.find_elements(By.CSS_SELECTOR, "div.sc-1nre5ec-1 > a".format(10))

        original_window = driver.current_window_handle

        j = 0
        for i in app_link:


            driver.execute_script("window.open('{}')".format(i.get_attribute("href")))
            driver.switch_to.window(driver.window_handles[1])

            #Recupperer Titre Annonce
            titre = ""
            try:
                titre = driver.find_element(By.CSS_SELECTOR, "div.sc-1g3sn3w-9 > h1.sc-1g3sn3w-12".format(10)).text
            except NoSuchElementException:
                titre = ""

            #Recupperer Prix
            prix = ""
            try:
                prix = driver.find_element(By.CSS_SELECTOR, "div.sc-1g3sn3w-10 > p.sc-1x0vz2r-0".format(10)).text
            except NoSuchElementException:
                prix = ""

            #Recupperer Ville
            ville = ""
            try:
                ville = driver.find_element(By.CSS_SELECTOR, "div.sc-1g3sn3w-7 > span.sc-1x0vz2r-0".format(10)).text
            except NoSuchElementException:
                ville = ""


            #Recupperer liste elements

            elements_dict = {
                "Type": "",
                "Secteur": "",
                "Étage": "",
                "Frais": "",
                "Salons": "",
                "Âge": "",
                "Salons":"",
                "Surface": "",
                "SalleDeBainTitleID": "",
                "ChambresTitleID": ""
            }
            elements = []
            try:
                elements = driver.find_elements(By.CSS_SELECTOR, "ol.sc-qmn92k-3 > li.sc-qmn92k-1 > span.sc-1x0vz2r-0".format(10))
                for key in elements_dict:
                    for i in range(0,len(elements)):
                        if elements[i].text.startswith(key):
                            elements_dict[key] = elements[i+1].text
                            break  # Exit the loop once a matching element is found
            except NoSuchElementException:
                elements = []

            icons = []
            values = []
            try:
                icons = driver.find_elements(By.CSS_SELECTOR, "div.sc-6p5md9-2 > div.sc-wdregf-0 > svg.av-icon".format(10))
                values = driver.find_elements(By.CSS_SELECTOR, "div.sc-6p5md9-2 > span.kQHNss".format(10))

                for key in elements_dict:
                    for i in range(0,len(icons)):
                        if icons[i].get_attribute("aria-labelledby").startswith(key):
                            elements_dict[key] = values[i].text
                            break  # Exit the loop once a matching element is found
            except NoSuchElementException:
                icons = []
                alues = []

            #Recupperer Description
            description = ""
            try:
                description = driver.find_element(By.CSS_SELECTOR, "div.sc-1g3sn3w-16 > p.sc-ij98yj-0".format(10)).text
            except NoSuchElementException:
                description = ""

             #ecrire dans un fichier csv
            csvWriter.writerow([titre, prix, ville, elements_dict["Type"], elements_dict["Secteur"], elements_dict["Étage"], elements_dict["Âge"], elements_dict["Surface"], elements_dict["Salons"], elements_dict["ChambresTitleID"], elements_dict["SalleDeBainTitleID"], elements_dict["Frais"], description ])
            j += 1
            driver.close()
            driver.switch_to.window(original_window)
    except StaleElementReferenceException:
        next = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div.sc-2y0ggl-0 > a.sc-1cf7u6r-0")))
        #driver.get(url+str(i))
        

    

In [3]:
import pandas as pd

df = pd.read_csv("Appartements.csv")

In [4]:
df.head()

,Titre,Prix,Ville,Type,Secteur,Etage,Age du bien,Surface habitable,NB Salons,NB Chambres,NB Salles de Bain,Frais de syndic / mois,Description
0,Appartement 165m² narjiss à El Jadida,1 160 000 DH,El Jadida,"Appartements, à vendre",Narjiss,1,NaN,229,3.0,6.0,2.0,NaN,un appart 115m2 et garçonnière 50m2 à vendre\n...
1,Appartement à vendre 62 m² à El Jadida,360 000 DH,El Jadida,"Appartements, à vendre",Autre secteur,2,6-10 ans,62,1.0,2.0,1.0,NaN,un appart 62m2 à vendre\nappart au quartier hi...
2,Appartement hay essalam 92 m² à El Jadida,750 000 DH,El Jadida,"Appartements, à vendre",Hay Essalam,2,6-10 ans,92,1.0,3.0,1.0,NaN,un appart 92m2 à vendre\nappart au quartier sa...
3,Appartement 110m² 2ème étg. résd. fermée Corniche,2 100 000 DH,Casablanca,"Appartements, à vendre",Bourgogne,2,1-5 ans,136,1.0,2.0,2.0,500.0,Appartement moderne 110m² au 2ème étg. en rési...
4,app a vendre,7 400 000 DH,El Mansouria,"Appartements, à vendre",Toute la ville,2,Neuf,60,1.0,1.0,1.0,450.0,vendre un belle appartement vie sur piscine et...
